In [2]:
#Chiefs Data Scrape using http://pro-football-reference.com


import pandas as pd
from bs4 import BeautifulSoup as bs
import sys 
import os
from urllib.request import urlopen
import numpy as np
import re
import os


import time


#change max_columns in pandas

pd.set_option('max_columns', 50)

#start scipt clock
time1 = time.time()

print(sys.version)
print(os.getcwd())

3.8.8 (default, Apr 13 2021, 12:59:45) 
[Clang 10.0.0 ]
/Users/drew/Documents/git_repos/chiefs_attendance_model


In [9]:
#determine how many years of data you would like to scrape

years = list(range(2005,2020))

years[-1]

2019

In [10]:
print(sys.version)

3.8.8 (default, Apr 13 2021, 12:59:45) 
[Clang 10.0.0 ]


In [11]:
#single year pull
url = 'https://www.pro-football-reference.com/teams/kan/2017.htm'

In [12]:
page = urlopen(url)
soup = bs(page)
table = soup.find_all("table")
table = table[1]

In [14]:
#pull for all years
for year in years:
  time1 = time.time()
  #create a lag so program run at a human's pace
  time.sleep(30)
  if year == years[0]:
      predf = dict()
      week = 0
      urls = 'https://www.pro-football-reference.com/teams/kan/'
      end = '.htm'
      url = urls+str(year)+end
      page = urlopen(url).read()
      soup = bs(page)
      table = soup.find_all("table")
      table = table[1]
      rows = table.find_all('tr')
      for row in rows:
          a = row.find_all('td')
          for rowrow in a:
              b = rowrow.text.strip().encode()
              c = b.decode('utf-8')
              ff = rowrow['data-stat']
              if ff in predf:
                  predf[ff].append(c)
              else:
                  predf[ff] = [c]
      df1 = pd.DataFrame.from_dict(predf)
      aa = []
      for index,row in df1.iterrows():
          aa.append(index + 1)
      df1['week'] = aa
      df1['season'] = year
  else:
      predf = dict()
      urls = 'https://www.pro-football-reference.com/teams/kan/'
      end = '.htm'
      url = urls+str(year)+end
      page = urlopen(url).read()
      soup = bs(page)
      table = soup.find_all("table")
      table = table[1]
      rows = table.find_all('tr')
      for row in rows:
          a = row.find_all('td')
          for rowrow in a:
              b = rowrow.text.strip().encode()
              c = b.decode('utf-8')
              ff = rowrow['data-stat']
              if ff in predf:
                  predf[ff].append(c)
              else:
                  predf[ff] = [c]
      df2 = pd.DataFrame.from_dict(predf)
      aa = []
      for index,row in df2.iterrows():
          aa.append(index + 1)
      df2['week'] = aa
      df2['season'] = year
      df1 = pd.concat([df1,df2],axis = 0,ignore_index=True)
  
  print(year,round(time.time() - time1, 2))
            

2005 4.16
2006 4.62
2007 4.67
2008 4.63
2009 5.11
2010 4.48
2011 4.12
2012 4.37
2013 4.5
2014 4.16
2015 4.6
2016 4.19
2017 4.41
2018 4.79
2019 4.37


In [15]:
#rearrange columns so season and week are first
df1 = pd.concat([df1[['season','week']],df1.iloc[:,:-2]],axis = 1)

In [16]:
df1.iloc[176:178,:]

,season,week,game_day_of_week,game_date,game_time,boxscore_word,game_outcome,overtime,team_record,game_location,opp,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,exp_pts_off,exp_pts_def,exp_pts_st
176,2015,1,Sun,September 13,1:02PM ET,boxscore,W,,1-0,@,Houston Texans,27,20,19,330,233,97,,24,396,298,98,2,1.05,4.01,2.24
177,2015,2,Thu,September 17,8:29PM ET,boxscore,L,,1-1,,Denver Broncos,24,31,20,314,167,147,5,21,299,238,61,1,-9.40,6.63,-5.00


In [17]:
#add feature for playoff games
df1['playoff_game'] = 'no'
aa = []
for index, row in df1.iterrows():
    if row['game_location'] == '@':
        aa.append('away')
    else:
        aa.append('home')
df1['game_location'] = aa

In [18]:
for index, row in df1.iterrows():
    if row['week'] > 17:
        df1.loc[index,'week'] = df1.loc[index,'week'] - 1
        df1.loc[index,'playoff_game'] = 'yes'
        

In [19]:
df1 = df1[df1['game_date'] != 'Playoffs']
del[df1['boxscore_word']]

In [20]:
print('total script time', round((time.time() - time1) / 60,2),'minutes')

total script time 0.39 minutes


In [21]:
#get attendance html

url = 'https://www.pro-football-reference.com/years/2006/attendance.htm'
page = urlopen(url).read()

soup = bs(page)

In [22]:
table = soup.find("table", {'id' : 'attendance'})

In [23]:
predf = dict()
rows = table.find_all('tr')


for row in rows:
  b = row.find_all('th')[0].text.strip()
  if b == 'Tm':
    pass
  elif 'team' not in predf:
    print(b)
    predf['team'] = [b]
  else:
    predf['team'].append(b)

for row in rows:
  a = row.find_all('td')
  for rowrow in a:
      b = rowrow.text.strip().encode()
      c = b.decode('utf-8')
      ff = rowrow['data-stat']
      if ff in predf:
          predf[ff].append(c)
      else:
          predf[ff] = [c]

Arizona Cardinals


In [24]:
a = pd.DataFrame(predf)

aa = ['team']

for n in list(range(1,18)):
  aa.append('week'+str(n))
  
b = a[a['team'] == 'Kansas City Chiefs'][aa].transpose().reset_index()
 
b.columns = ['week','attendance']


b = b.iloc[1:,:].reset_index()

del[b['index']]

b.head()

,week,attendance
0,week1,"77,956"
1,week2,"76,786"
2,week3,Bye
3,week4,"77,609"
4,week5,"63,445"


In [27]:
#make dataset of all years for attendance

for year in years:
  print(year)
  time.sleep(10) 
  if year == years[0]:
    url = 'https://www.pro-football-reference.com/years/' + str(year) + '/attendance.htm'
    page = urlopen(url).read()
    soup = bs(page)
    table = soup.find("table", {'id' : 'attendance'})
    predf = dict()
    rows = table.find_all('tr')

    for row in rows:
      b = row.find_all('th')[0].text.strip()
      if b == 'Tm':
        pass
      elif 'team' not in predf:
        #print(b)
        predf['team'] = [b]
      else:
        predf['team'].append(b)

    for row in rows:
      a = row.find_all('td')
      for rowrow in a:
          b = rowrow.text.strip().encode()
          c = b.decode('utf-8')
          ff = rowrow['data-stat']
          if ff in predf:
              predf[ff].append(c)
          else:
              predf[ff] = [c]
    a = pd.DataFrame(predf)

    aa = ['team']

    for n in list(range(1,18)):
      aa.append('week'+str(n))

    attend = a[a['team'] == 'Kansas City Chiefs'][aa].transpose().reset_index()

    attend.columns = ['week','attendance']


    attend = attend.iloc[1:,:].reset_index()

    del[attend['index']]
    attend['year'] = year
  else:
    url = 'https://www.pro-football-reference.com/years/' + str(year) + '/attendance.htm'
    page = urlopen(url).read()
    soup = bs(page)
    table = soup.find("table", {'id' : 'attendance'})
    predf = dict()
    rows = table.find_all('tr')

    for row in rows:
      b = row.find_all('th')[0].text.strip()
      if b == 'Tm':
        pass
      elif 'team' not in predf:
        #print(b)
        predf['team'] = [b]
      else:
        predf['team'].append(b)

    for row in rows:
      a = row.find_all('td')
      for rowrow in a:
          b = rowrow.text.strip().encode()
          c = b.decode('utf-8')
          ff = rowrow['data-stat']
          if ff in predf:
              predf[ff].append(c)
          else:
              predf[ff] = [c]
    a = pd.DataFrame(predf)

    aa = ['team']

    for n in list(range(1,18)):
      aa.append('week'+str(n))

    attend1 = a[a['team'] == 'Kansas City Chiefs'][aa].transpose().reset_index()

    attend1.columns = ['week','attendance']


    attend1 = attend1.iloc[1:,:].reset_index()

    del[attend1['index']]
    attend1['year'] = year
    attend = pd.concat([attend,attend1], axis = 0)
    




2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


In [28]:
attend.head()

,week,attendance,year
0,week1,"78,014",2005
1,week2,"62,273",2005
2,week3,"76,381",2005
3,week4,"78,742",2005
4,week5,Bye,2005


In [29]:
#make unique key out of year and week

aa = []

for index,row in attend.iterrows():
  year = str(row['year'])
  week = re.sub('[a-z]', '', row['week'])
  year_week = year + week
  aa.append(year_week)
  
attend['year_week'] = aa

In [30]:
attend.head()

,week,attendance,year,year_week
0,week1,"78,014",2005,20051
1,week2,"62,273",2005,20052
2,week3,"76,381",2005,20053
3,week4,"78,742",2005,20054
4,week5,Bye,2005,20055


In [31]:
df1.head()

,season,week,game_day_of_week,game_date,game_time,game_outcome,overtime,team_record,game_location,opp,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,exp_pts_off,exp_pts_def,exp_pts_st,playoff_game
0,2005,1,Sun,September 11,1:00PM ET,W,,1-0,home,New York Jets,27,7,23,389,191,198,1,19,390,333,57,3,12.26,11.60,-5.04,no
1,2005,2,Sun,September 18,8:30PM ET,W,,2-0,away,Oakland Raiders,23,17,17,354,229,125,1,16,327,255,72,2,3.72,-1.52,1.18,no
2,2005,3,Mon,September 26,9:07PM ET,L,,2-1,away,Denver Broncos,10,30,18,285,211,74,1,24,358,137,221,,-1.23,-19.32,-2.42,no
3,2005,4,Sun,October 2,4:15PM ET,L,,2-2,home,Philadelphia Eagles,31,37,19,353,209,144,4,25,396,368,28,2,-3.72,-8.01,11.24,no
4,2005,5,,,,,,,home,Bye Week,,,,,,,,,,,,,,,,no


In [32]:
#create year_week for main dataframe

aa = [] 

for index,row in df1.iterrows():
  year = str(row['season'])
  week = str(row['week'])
  aa.append(year + week)
  
df1['year_week'] = aa

In [33]:
df = df1.merge(attend[['attendance','year_week']], on = 'year_week')

#delete matching key
del[df['year_week']]

In [34]:
df.tail()

,season,week,game_day_of_week,game_date,game_time,game_outcome,overtime,team_record,game_location,opp,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,exp_pts_off,exp_pts_def,exp_pts_st,playoff_game,attendance
250,2019,13,Sun,December 1,4:25PM ET,W,,8-4,home,Oakland Raiders,40,9,22,259,163,96,,19,332,210,122,3,14.17,3.44,11.19,no,"73,548"
251,2019,14,Sun,December 8,4:25PM ET,W,,9-4,away,New England Patriots,23,16,20,346,271,75,2,17,278,184,94,1,-1.14,5.79,1.25,no,"65,878"
252,2019,15,Sun,December 15,1:00PM ET,W,,10-4,home,Denver Broncos,23,3,27,419,327,92,1,15,251,199,52,1,15.31,9.67,-4.87,no,"73,257"
253,2019,16,Sun,December 22,8:20PM ET,W,,11-4,away,Chicago Bears,26,3,25,350,244,106,,18,234,133,101,,14.74,6.38,1.09,no,"62,213"
254,2019,17,Sun,December 29,1:00PM ET,W,,12-4,home,Los Angeles Chargers,31,21,16,336,174,162,1,25,366,258,108,2,13.59,-9.18,7.29,no,"73,680"


In [36]:
df.to_csv('chiefs_data2019.csv',index = False)